# Assigment 4

In [7]:
#! pip install lxml
#! pip install unidecode

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

## Scrape all the results from "Extranjero" and get information from all tabs

# Click in all "Continentes", "Países" and "Estados" and extract table

In [2]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
# go to the link
driver.get( 'https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E' )
# Maximize window
driver.maximize_window()
time.sleep(5)   

# Store all_tables
all_tables = {}
# All selenium objects in CONTINENTES select
cont = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
# Get number of total options
num_cont_options = len( cont.options )

for cont_idx in range( num_cont_options ):
    
    # Get again all CONTINENTES since HTML is refreshing
    # all elements
    
    cont = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    continente = cont.options[ cont_idx ]
    
    # Get CONTINENTE name
    cont_name = continente.text
    
    if cont_name != "--TODOS--" :
        
        # click on CONTINENTE
        continente.click()
        
        # Get all elements of PAÍS
        pais= Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
        num_pais_options = len( pais.options )
        
        for pais_idx in range( num_pais_options ):
            
            # Get again all PAISES since HTML is refreshing
            # all elements
            pais = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            paises = pais.options[ pais_idx ]
                
            # Get PAISES name
            pais_name = paises.text
            
            if pais_name != "--TODOS--" :
                
                # click on PAISES
                paises.click()
                
                # Get all elements from ESTADO
                est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_est_options = len( est.options )
                
                for est_idx in range( num_est_options ):
                    
                    # Get again all ESTADO since HTML is refreshing
                    # all elements
                    est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                    estados = est.options[ est_idx ]
                    
                    # Get ESTADO name
                    est_name = estados.text
                    
                    if est_name != "--TODOS--" :
                        
                        # click on ESTADO
                        estados.click()
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                                                                  
                        ## Get table of presidential votes
                        # Get html at this point
                        table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( "ORGANIZACIONES POLÍTICAS", "political_parties") \
                                              .str.replace( "TOTAL", "total") \
                                              .str.replace( "%VÁLIDOS", "share_valid") \
                                              .str.replace( "%EMITIDOS", "share_emited") \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'continent' ] = cont_name
                        table_clean[ 'country' ]   = pais_name
                        table_clean[ 'state' ]     = est_name
                        table_clean[ 'ubigeo' ]    = ubigeo
                        table_clean['link']        = driver.current_url
                        
                        # store tables
                        all_tables [ubigeo] = table_clean



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Robert\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-2-5d80b0e6c405>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )
<ipython-input-2-5d80b0e6c405>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cont = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select"}
  (Session info: chrome=96.0.4664.93)
Stacktrace:
Backtrace:
	Ordinal0 [0x01246903+2517251]
	Ordinal0 [0x011DF8E1+2095329]
	Ordinal0 [0x010E2848+1058888]
	Ordinal0 [0x0110D448+1233992]
	Ordinal0 [0x0110D63B+1234491]
	Ordinal0 [0x01137812+1406994]
	Ordinal0 [0x0112650A+1336586]
	Ordinal0 [0x01135BBF+1399743]
	Ordinal0 [0x0112639B+1336219]
	Ordinal0 [0x011027A7+1189799]
	Ordinal0 [0x01103609+1193481]
	GetHandleVerifier [0x013D5904+1577972]
	GetHandleVerifier [0x01480B97+2279047]
	GetHandleVerifier [0x012D6D09+534521]
	GetHandleVerifier [0x012D5DB9+530601]
	Ordinal0 [0x011E4FF9+2117625]
	Ordinal0 [0x011E98A8+2136232]
	Ordinal0 [0x011E99E2+2136546]
	Ordinal0 [0x011F3541+2176321]
	BaseThreadInitThunk [0x7706FA29+25]
	RtlGetAppContainerNamedObjectPath [0x779F7A9E+286]
	RtlGet

In [17]:
final_data = pd.concat( all_tables.values()).reset_index( drop = True )

In [18]:
final_data

,NaN,total,share_valid,share_emited,continent,country,state,ubigeo,link
0,PARTIDO NACIONALISTA PERUANO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
2,PARTIDO MORADO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
3,PERU PATRIA SEGURA,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
4,VICTORIA NACIONAL,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
...,...,...,...,...,...,...,...,...,...
3826,ALIANZA PARA EL PROGRESO,1,0.699%,0.617%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3827,TOTAL DE VOTOS VÁLIDOS,143,100.000%,88.272%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3828,VOTOS EN BLANCO,15,NaN,9.259%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3829,VOTOS NULOS,4,NaN,2.469%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...


In [19]:
final_data.to_excel( r'example_round.xlsx' , index = False )